In [ ]:
import glob
import pandas as pd
import gensim
import spacy
from pprint import pprint

from utils.helper import preprocess_tweet, sentence_to_words, lemmatization

In [3]:
# fintech_path = './data/fintech/'
# fintech_folders = glob.glob(f"{fintech_path}*/", recursive=True)
# fintech = [path.split('/')[-2] for path in fintech_folders]
# fintech_files = glob.glob(fintech_path + "**/*.csv")

# nbfc_path = './data/nbfc/'
# nbfc_folders = glob.glob(f"{nbfc_path}*/", recursive=True)
# nbfc = [path.split('/')[-2] for path in nbfc_folders]
# nbfc_files = glob.glob(nbfc_path + "**/*.csv")

In [4]:
# fintech_list = []
# for comp, file in zip(fintech, fintech_files):
#     temp = pd.read_csv(file, engine='python')
#     temp['company'] = comp
#     fintech_list.append(temp)

# fintech_df = pd.concat(fintech_list, axis=0, ignore_index=True)

# fintech_df.to_hdf('./data/fintech/fintech.h5', key='fintech')

# nbfc_list = []
# for comp, file in zip(nbfc, nbfc_files):
#     temp = pd.read_csv(file, engine='python')
#     temp['company'] = comp
#     nbfc_list.append(pd.read_csv(file, engine='python'))

# nbfc_df = pd.concat(nbfc_list, axis=0, ignore_index=True)

In [5]:
fintech_df = pd.read_hdf('./data/fintech/fintech.h5', key='fintech')

In [6]:
fintech_df.head()

,tweet,conversation_id,date,hashtags,id,inReplyToTweetId,reply_to,language,likes_count,media,...,user_status_count,location,name,description,verified,url,user_id,username,company,preprocessed
0,@nsitharaman @RBI @Lendingkart My loan Id LAI-...,1344359252224143360,2020-12-30 19:06:37+00:00,NaN,1344359252224143360,NaN,NaN,en,0.0,NaN,...,172.0,"Bilaspur, India",ABHI TRAVELS AND TATA SKY D2H SALES,"Car rental and Dth sales for call 9399033577,8...",False,https://youtu.be/O2Z9TfA4yno,2.810484e+09,D2hS30,lendingkart,my loan id lai00091882 please reverse my c...
1,@RBI @Lendingkart @FinMinIndia My loan Id LAI-...,1344357541707354113,2020-12-30 18:59:49+00:00,NaN,1344357541707354113,NaN,NaN,en,0.0,NaN,...,172.0,"Bilaspur, India",ABHI TRAVELS AND TATA SKY D2H SALES,"Car rental and Dth sales for call 9399033577,8...",False,https://youtu.be/O2Z9TfA4yno,2.810484e+09,D2hS30,lendingkart,my loan id lai00091882 please reverse my c...
2,@Lendingkart I have not recvd any thing from y...,1313373475407491073,2020-12-30 15:21:38+00:00,NaN,1344302632177831937,1.343920e+18,https://twitter.com/Lendingkart,en,0.0,NaN,...,929.0,NaN,Raajeev Kotak,NaN,False,NaN,9.756444e+08,RaajeevKotak,lendingkart,i have not recvd any thing from your end kind...
3,@Lendingkart dear I was asking for restructuri...,1344280074103582721,2020-12-30 13:51:59+00:00,"['operationhaftaavasooli', 'Hallabol', 'loanma...",1344280074103582721,NaN,NaN,en,0.0,[Photo(previewUrl='https://pbs.twimg.com/media...,...,4280.0,India,Shivansh,"#SoftwareEngineer ,# I.T. Consultant #Socialwo...",False,http://shivaconsults.com,1.244710e+18,Shivans44729084,lendingkart,dear i was asking for restructuring you are f...
4,"@AKHILESHU07 Hello, Kindly DM us or share your...",1343785976942882818,2020-12-30 13:51:31+00:00,NaN,1344279954549161990,1.344264e+18,https://twitter.com/AKHILESHU07,en,0.0,NaN,...,5022.0,"Ahmedabad, Gujarat",Lendingkart,Simplifying MSME finance. Business Loans Bole ...,False,https://tinyurl.com/45ummdxf,2.912551e+09,Lendingkart,lendingkart,hello kindly dm us or share your registered c...


In [7]:
# Preprocess tweets
fintech_df['preprocessed'] = fintech_df['tweet'].apply(preprocess_tweet)

In [9]:
# Create words data
fintech_data = fintech_df['preprocessed'].values.tolist()
# fintech_data_words = list(sentence_to_words(fintech_data))
fintech_data_words = list(sentence_to_words(fintech_data))

In [10]:
# Build bigram and trigram models
bigram = gensim.models.phrases.Phrases(fintech_data_words, min_count=5, threshold=10, connector_words=gensim.models.phrases.ENGLISH_CONNECTOR_WORDS)
trigram = gensim.models.phrases.Phrases(bigram[fintech_data_words], threshold=100)

bigram_model = gensim.models.phrases.Phraser(bigram)
trigram_model = gensim.models.phrases.Phraser(trigram)

In [11]:
# Remove stopwords
fintech_df['preprocessed'] = fintech_df['preprocessed'].apply(gensim.parsing.preprocessing.remove_stopwords)

In [12]:
fintech_df['sep_words'] = fintech_df['preprocessed'].apply(lambda x: list(sentence_to_words([x]))[0])

In [13]:
fintech_df['bigram'] = fintech_df['sep_words'].apply(lambda x: bigram_model[x])

In [14]:
fintech_df['trigram'] = fintech_df['bigram'].apply(lambda x: [trigram_model[x]])

In [18]:
id2word = gensim.corpora.Dictionary(fintech_df['bigram'].values.tolist())

In [19]:
corpus = [id2word.doc2bow(text) for text in fintech_df['bigram'].values.tolist()]

In [22]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                        id2word=id2word,
                                        num_topics=10,
                                        random_state=42,
                                        chunksize=100,
                                        passes=10,
                                        per_word_topics=True,
                                        workers=4)

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]